In this Notebook we want to create train, test and validation datasets of flights for our landing time prediction model. We are going 

In [23]:
from traffic.core import Traffic
from preprocessing import get_complete_flights, preprocess_traffic
import h5py
import numpy as np
import pandas as pd
import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [24]:
def generate_batches(files, batch_size):
    counter = 0

    while True:
        fname = files[counter]

        counter = (counter + 1) % len(files)
        frame = pd.read_csv(INPUT_DATA_DIR + fname)

        # here is your preprocessing

        input = # so you init input here somehow
        output = # so you init output here

        for local_index in range(0, input.shape[0], batch_size):
            input_local = input[local_index:(local_index + batch_size)]
            output_local = output[local_index:(local_index + batch_size)]

            yield input_local, output_local

SyntaxError: invalid syntax (3272259913.py, line 12)

In [30]:
X = None
y = None
def data_generator_flights():
    months = ["01","05"]
    while True:
        first = True
        for month in months:
            file = "Frankfurt_LH_22" + month + ".h5"
            for day in range(1, 3):

                if month in ["06", "09", "11"] and day ==31:
                    continue
                str_day = str(day) if day > 9 else "0"+str(day)
                key = "LH_22"+month+str_day
                with h5py.File(file, 'r') as f:
                    print(key)
                    if key in list(f.keys()):
                        flights = Traffic.from_file(file, key = key, parse_dates=["day","firstseen", "hour", "last_position","lastseen","timestamp"])
                    else:
                        continue
                    try:
                        df_flights = preprocess_traffic(flights)
                    except:
                        continue
                    df = df_flights[["distance", "altitude", "geoaltitude", "arrival_time", "timestamp", "vertical_rate", "groundspeed"]].dropna()
                    df_sample = df.sample(frac=0.05)
                    if not first:
                        df_train = pd.concat([df_sample, df_train])
                    else:
                        df_train = df_sample
                        first = False

        X = np.array(df_train[["distance", "altitude", "geoaltitude", "vertical_rate", "groundspeed"]])  # we only take the feature
        y = seconds_till_arrival(df_train)
        scaler = StandardScaler()
        scaler = scaler.fit(X)
        X_scaled = scaler.transform(X)
        
        X = X_scaled


In [26]:
def build_sequential(lr, input_dims, output_dims, layerSizes, activation, loss):
    model = keras.Sequential()
    model.add (keras.layers.Input(shape=input_dims))
    for size in layerSizes:
        model.add(keras.layers.Dense(size))
        model.add(keras.layers.LeakyReLU(alpha=0.05))

    model.add(keras.layers.Dense(output_dims,activation=activation))

    model.compile(optimizer=Adam(learning_rate=lr), loss=loss)

    return model

In [27]:
#create feature as seconds till landing
def seconds_till_arrival(flights_data: pd.DataFrame):
    time_till_arrival = flights_data["arrival_time"]- flights_data["timestamp"]
    seconds = time_till_arrival.dt.total_seconds()
    return seconds